<a href="https://colab.research.google.com/github/MarianaDuartee/ProjetoFinal/blob/main/2_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PYSPARK

### INSTALANDO DEPENDECIAS

In [ ]:
!pip install pyspark
!pip install gcsfs
!pip install pandas
!pip install pandera

### IMPORTANDO BIBLIOTECAS, ABRINDO SPARKSESSION E CONFIGURANDO CHAVE DE SERVIÇO

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Window
from google.cloud import storage

import pyspark
import pyspark.sql.functions as F
import pandas as pd
import pandera as pa
import os
import gcsfs

In [9]:
spark = SparkSession.builder\
.master('local')\
.appName('Projeto_Final')\
.config('spark.ui.enable', 'true')\
.config('spark.ui.port', '4050')\
.getOrCreate()

spark

serviceaccount = '/content/soulcode-projeto-final-4b88bea6e07a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

spark

### IMPORTANDO, VALIDANDO, EXTRAINDO INFORMAÇÕES E ANALISANDO DATAFRAMES

DATAFRAME 1

In [10]:
# ARQUIVO 1 - Lendo e validando importanção
file_path_1 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_despesas_normalizado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_1):
    data = pd.read_csv(file_path_1, sep=',', encoding='UTF-8', header=0, index_col=False)

    # Validando esquema, coluna e tipo de dado
    schema = pa.DataFrameSchema(columns={"UF": pa.Column(str, pa.Check(str)),
                                         "Despesas2016": pa.Column(float, pa.Check(lambda s: s > 100.000), nullable=True),
                                         "Despesas2017": pa.Column(float, pa.Check(lambda s: s > 100.000), nullable=True),
                                         "Despesas2018": pa.Column(float, pa.Check(lambda s: s > 100.000), nullable=True),
                                         "Despesas2019": pa.Column(float, pa.Check(lambda s: s > 100.000), nullable=True),
                                         "Despesas2020": pa.Column(float, pa.Check(lambda s: s > 100.000), nullable=True),
                                         "Variacao%": pa.Column(float, pa.Check(lambda s: s != 0), nullable=True),
                                         "Previsao2021|Media": pa.Column(float, pa.Check(lambda s: s > 100.000), nullable=True)                               
                                         })
    
    schema.validate(data)

df_1 = spark.createDataFrame(data)
df_1.dtypes
df_1.show(10, truncate=False)

+---------+---------------+---------------+---------------+---------------+---------------+---------+--------------------+
|UF       |Despesas2016   |Despesas2017   |Despesas2018   |Despesas2019   |Despesas2020   |Variacao%|Previsao2021|Media  |
+---------+---------------+---------------+---------------+---------------+---------------+---------+--------------------+
|Acre     |4.9853469586E8 |5.6836139244E8 |6.273508495E8  |6.9257020034E8 |4.9375329505E8 |8.5      |5.761140866380001E8 |
|Amapá    |4.7532329175E8 |5.0022689486E8 |4.8922797864E8 |6.0636632529E8 |7.5225249884E8 |52.3     |5.646793978759999E8 |
|Amazonas |1.60473221563E9|1.57895665667E9|1.86266890656E9|2.10845995454E9|2.10144665649E9|60.4     |1.851252877978E9    |
|Pará     |2.55174076992E9|2.57991598234E9|2.90320019444E9|2.94344753592E9|2.96655339806E9|58.0     |2.7889715761359997E9|
|Rondônia |8.7487402427E8 |8.9586766443E8 |9.0325385747E8 |8.5397099952E8 |9.9904393587E8 |-14.7    |9.054020963119999E8 |
|Roraima  |4.089

DATAFRAME 2

In [11]:
file_path_2 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencias_normalizado.json'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_2):
    data = pd.read_json(file_path_2, encoding='UTF-8')

# Validando esquema, coluna e tipo de dado
    schema = pa.DataFrameSchema(columns={"UF": pa.Column(str, pa.Check(str)),
                                         "TipoCrime": pa.Column(str, pa.Check(str), nullable=True),
                                         "Ano": pa.Column(int, pa.Check(lambda s: s >= 2015), nullable=True),
                                         "Mes": pa.Column(str, pa.Check(str), nullable=True),
                                         "Ocorrencias": pa.Column(int, pa.Check(lambda s: s >= 0), nullable=True),                          
                                         })
    
    schema.validate(data)
    
df_2 = spark.createDataFrame(data)
df_2.dtypes
df_2.show(10, truncate=False)

+----+-----------------------------------+----+---------+-----------+
|UF  |TipoCrime                          |Ano |Mes      |Ocorrencias|
+----+-----------------------------------+----+---------+-----------+
|Acre|Estupro                            |2021|janeiro  |39         |
|Acre|Furto de veículo                   |2021|janeiro  |55         |
|Acre|Homicídio doloso                   |2021|janeiro  |14         |
|Acre|Lesão corporal seguida de morte    |2021|janeiro  |0          |
|Acre|Roubo a instituição financeira     |2021|janeiro  |0          |
|Acre|Roubo de carga                     |2021|janeiro  |0          |
|Acre|Roubo de veículo                   |2021|janeiro  |99         |
|Acre|Roubo seguido de morte (latrocínio)|2021|janeiro  |1          |
|Acre|Tentativa de homicídio             |2021|janeiro  |10         |
|Acre|Estupro                            |2021|fevereiro|23         |
+----+-----------------------------------+----+---------+-----------+
only showing top 10 

DATAFRAME 3

In [12]:
file_path_3 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_vitimas_normalizado.json'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_3):
    data = pd.read_json(file_path_3, encoding='UTF-8')

    # Validando esquema, coluna e tipo de dado
    schema = pa.DataFrameSchema(columns={"UF": pa.Column(str, pa.Check(str)),
                                     "TipoCrime": pa.Column(str, pa.Check(str), nullable=True),
                                     "Ano": pa.Column(int, pa.Check(lambda s: s >= 2015), nullable=True),
                                     "Mes": pa.Column(str, pa.Check(str), nullable=True),
                                     "SexoVitima": pa.Column(str, pa.Check(str), nullable=True),
                                     "Vitimas": pa.Column(int, pa.Check(lambda s: s >= 0), nullable=True),                      
                                    })
    
    schema.validate(data)

df_3 = spark.createDataFrame(data)
df_3.dtypes
df_3.show(10, truncate=False)

+----+----------------+----+---------+----------+-------+
|UF  |TipoCrime       |Ano |Mes      |SexoVitima|Vitimas|
+----+----------------+----+---------+----------+-------+
|Acre|Homicídio doloso|2021|janeiro  |Feminino  |1      |
|Acre|Homicídio doloso|2021|janeiro  |Masculino |13     |
|Acre|Homicídio doloso|2021|janeiro  |Sexo NI   |0      |
|Acre|Homicídio doloso|2021|fevereiro|Feminino  |4      |
|Acre|Homicídio doloso|2021|fevereiro|Masculino |12     |
|Acre|Homicídio doloso|2021|fevereiro|Sexo NI   |0      |
|Acre|Homicídio doloso|2021|março    |Feminino  |2      |
|Acre|Homicídio doloso|2021|março    |Masculino |24     |
|Acre|Homicídio doloso|2021|março    |Sexo NI   |0      |
|Acre|Homicídio doloso|2021|abril    |Feminino  |2      |
+----+----------------+----+---------+----------+-------+
only showing top 10 rows



DATAFRAME 4

In [13]:
file_path_4 = 'gs://data_lake_ingest_data/1_input/Frequencia_escolar_UF.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_4):
    data = pd.read_csv(file_path_4, encoding='UTF-8')

    data['Média'] = data['Média'].str.replace(',', '.')
    data['Média'] = data['Média'].astype(float)

    # Validando esquema, coluna e tipo de dado
    schema = pa.DataFrameSchema(columns={"UF": pa.Column(str, pa.Check(str)),
                                        "Média": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                        "CV (%)": pa.Column(str, pa.Check(str), nullable=True),
                                        "Ano": pa.Column(int, pa.Check(lambda s: s >= 2016), nullable=True),                      
                                        })
    
    schema.validate(data)

df_4 = spark.createDataFrame(data)
df_4.dtypes
df_4.show(10, truncate=False)

+---------+-----+-------+----+
|UF       |Média|CV (%) |Ano |
+---------+-----+-------+----+
|Rondônia |11.3 |   0,9 |2019|
|Acre     |11.1 |   1,0 |2019|
|Amazonas |11.2 |   0,9 |2019|
|Roraima  |11.7 |   1,0 |2019|
|Pará     |10.5 |   0,8 |2019|
|Amapá    |11.5 |   1,3 |2019|
|Tocantins|11.4 |   0,9 |2019|
|Maranhão |10.8 |   0,5 |2019|
|Piauí    |11.1 |   1,0 |2019|
|Ceará    |11.2 |   0,5 |2019|
+---------+-----+-------+----+
only showing top 10 rows



DATAFRAME 5

In [14]:
file_path_5 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencia_vitimas_porAnoEstado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_5):
    data = pd.read_csv(file_path_5, sep=',', encoding='UTF-8')

    # Validando esquema, coluna e tipo de dado
    schema = pa.DataFrameSchema(columns={"UF": pa.Column(str, pa.Check(str)),
                                         "Ano": pa.Column(int, pa.Check(lambda s: s >= 2015), nullable=True),
                                         "TipoCrime": pa.Column(str, pa.Check(str), nullable=True),
                                         "Ocorrencias": pa.Column(int, pa.Check(lambda s: s >= 0), nullable=True),
                                         "Vitimas": pa.Column(int, pa.Check(lambda s: s >= 0), nullable=True),                      
                                        })
    
    schema.validate(data)

df_5 = spark.createDataFrame(data)
df_5.dtypes
df_5.show(10, truncate=False)

+----+----+-----------------------------------+-----------+-------+
|UF  |Ano |TipoCrime                          |Ocorrencias|Vitimas|
+----+----+-----------------------------------+-----------+-------+
|Acre|2016|Homicídio doloso                   |425        |354    |
|Acre|2017|Homicídio doloso                   |221        |232    |
|Acre|2017|Lesão corporal seguida de morte    |0          |3      |
|Acre|2017|Roubo seguido de morte (latrocínio)|10         |15     |
|Acre|2018|Homicídio doloso                   |396        |394    |
|Acre|2018|Roubo seguido de morte (latrocínio)|20         |20     |
|Acre|2019|Homicídio doloso                   |292        |298    |
|Acre|2019|Lesão corporal seguida de morte    |2          |3      |
|Acre|2019|Roubo seguido de morte (latrocínio)|14         |12     |
|Acre|2020|Homicídio doloso                   |271        |273    |
+----+----+-----------------------------------+-----------+-------+
only showing top 10 rows



DATAFRAME 6

In [15]:
file_path_6 = 'gs://data_lake_ingest_data/1_input/Tabela_frequencia_escolar.xlsx'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_6):
    data = pd.read_excel(file_path_6, header=0)

    # Validando esquema, coluna e tipo de dado
    schema = pa.DataFrameSchema(columns={"Grandes Regiões, Unidades da Federação e Municípios das Capitais": pa.Column(str, pa.Check(str)),
                                         "15 a 17 anos - 2019": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "15 a 17 anos - 2018": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "15 a 17 anos - 2017": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "15 a 17 anos - 2016": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),                      
                                         "Media 15 a 17 anos": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "18 a 24 anos - 2019": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "18 a 24 anos - 2018": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "18 a 24 anos - 2017": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "18 a 24 anos - 2016": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "média 18 a 24 anos": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "25 anos ou mais - 2019": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "25 anos ou mais - 2018": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "25 anos ou mais - 2017": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "25 anos ou mais - 2016":  pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "média 25 anos ou mais": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True)
                                        })
    
    schema.validate(data)
    
df_6 = spark.createDataFrame(data)
df_6.dtypes
df_6.show(10, truncate=False)

FileNotFoundError: ignored

DATAFRAME 7

In [17]:
file_path_7 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_taxa_analfabetismo_normalizado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_7):
    data = pd.read_csv(file_path_7, header=0)

    # Validando esquema, coluna e tipo de dado
    schema = pa.DataFrameSchema(columns={"UF": pa.Column(str, pa.Check(str), nullable=True),
                                         "Grupo_idade": pa.Column(str, pa.Check(str), nullable=True),
                                         "TaxaAnalfabetismo2016": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "TaxaAnalfabetismo2017": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),                      
                                         "TaxaAnalfabetismo2018": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                         "TaxaAnalfabetismo2019": pa.Column(float, pa.Check(lambda s: s <= 100.0), nullable=True),
                                        })
    
    schema.validate(data)

df_7 = spark.createDataFrame(data)
df_7.dtypes
df_7.show(10, truncate=False)

+---------+---------------+---------------------+---------------------+---------------------+---------------------+
|UF       |Grupo_idade    |TaxaAnalfabetismo2016|TaxaAnalfabetismo2017|TaxaAnalfabetismo2018|TaxaAnalfabetismo2019|
+---------+---------------+---------------------+---------------------+---------------------+---------------------+
|Rondônia |15 anos ou mais|6.6                  |7.2                  |6.5                  |6.4                  |
|Acre     |15 anos ou mais|13.1                 |12.1                 |12.1                 |11.7                 |
|Amazonas |15 anos ou mais|6.9                  |6.1                  |5.8                  |5.4                  |
|Roraima  |15 anos ou mais|6.6                  |6.0                  |6.0                  |5.0                  |
|Pará     |15 anos ou mais|9.2                  |8.6                  |8.8                  |8.4                  |
|Amapá    |15 anos ou mais|5.0                  |5.0                  |6

### NORMALIZANDO DATAFRAMES


DATAFRAME 1

In [18]:
df_1 = df_1.drop('Estimativa_2021', 'Previsao2021Media')
df_1 = df_1.withColumnRenamed('Media Anual', 'Estimativa_Despesa_2021')
df_1 = df_1.withColumnRenamed('Previsao2021|Media', 'Previsao2021_Media')

df_1 = df_1.withColumn("Despesas2017", F.round(df_1.Despesas2016.cast(FloatType()), 3))
df_1 = df_1.withColumn("Despesas2017", F.round(df_1.Despesas2017.cast(FloatType()), 3))
df_1 = df_1.withColumn("Despesas2018", F.round(df_1.Despesas2018.cast(FloatType()), 3))
df_1 = df_1.withColumn("Despesas2019", F.round(df_1.Despesas2019.cast(FloatType()), 3))
df_1 = df_1.withColumn("Despesas2020", F.round(df_1.Despesas2020.cast(FloatType()), 3))
df_1 = df_1.withColumn("Previsao2021_Media", F.round(df_1.Previsao2021_Media.cast(FloatType()), 3))

df_1.show()
df_1.dtypes

+-------------------+---------------+------------+------------+------------+------------+---------+------------------+
|                 UF|   Despesas2016|Despesas2017|Despesas2018|Despesas2019|Despesas2020|Variacao%|Previsao2021_Media|
+-------------------+---------------+------------+------------+------------+------------+---------+------------------+
|               Acre| 4.9853469586E8|4.98534688E8| 6.2735085E8| 6.9257018E8| 4.9375328E8|      8.5|       5.7611411E8|
|              Amapá| 4.7532329175E8|4.75323296E8|4.89227968E8| 6.0636634E8| 7.5225248E8|     52.3|       5.6467942E8|
|           Amazonas|1.60473221563E9|1.60473216E9|1.86266893E9| 2.1084599E9|2.10144666E9|     60.4|      1.85125286E9|
|               Pará|2.55174076992E9|2.55174067E9|2.90320026E9|2.94344755E9|2.96655334E9|     58.0|      2.78897152E9|
|           Rondônia| 8.7487402427E8| 8.7487405E8| 9.0325389E8| 8.5397101E8|  9.990439E8|    -14.7|       9.0540211E8|
|            Roraima| 4.0893419687E8|4.08934208E

[('UF', 'string'),
 ('Despesas2016', 'double'),
 ('Despesas2017', 'float'),
 ('Despesas2018', 'float'),
 ('Despesas2019', 'float'),
 ('Despesas2020', 'float'),
 ('Variacao%', 'double'),
 ('Previsao2021_Media', 'float')]

DATAFRAME 4

In [24]:
df_4 = df_4.drop("CV (%)")
df_4 = df_4.withColumnRenamed("Média", 'Media_Freq_Escolar')\
.orderBy("UF")

df_4.show(100, truncate=False)
df_4.dtypes

+-------------------+------------------+----+
|UF                 |Media_Freq_Escolar|Ano |
+-------------------+------------------+----+
|Acre               |10.6              |2016|
|Acre               |11.1              |2019|
|Acre               |10.9              |2018|
|Acre               |10.8              |2017|
|Alagoas            |10.4              |2018|
|Alagoas            |10.0              |2017|
|Alagoas            |9.7               |2016|
|Alagoas            |10.4              |2019|
|Amapá              |11.5              |2019|
|Amapá              |11.3              |2018|
|Amapá              |11.2              |2017|
|Amapá              |11.1              |2016|
|Amazonas           |10.8              |2017|
|Amazonas           |11.2              |2019|
|Amazonas           |10.5              |2016|
|Amazonas           |10.9              |2018|
|Bahia              |10.3              |2017|
|Bahia              |10.4              |2016|
|Bahia              |10.7         

[('UF', 'string'), ('Media_Freq_Escolar', 'double'), ('Ano', 'bigint')]

DATAFRAME 6

In [ ]:
# DROPANDO COLUNAS
df_6 = df_6.drop('15 a 17 anos - 2019', 
                 '15 a 17 anos - 2018',
                 '15 a 17 anos - 2017',
                 '15 a 17 anos - 2016',
                 '18 a 24 anos - 2019',
                 '18 a 24 anos - 2018',
                 '18 a 24 anos - 2017',
                 '18 a 24 anos - 2016',
                 '25 anos ou mais - 2019',
                 '25 anos ou mais - 2018',
                 '25 anos ou mais - 2017',
                 '25 anos ou mais - 2016')

# RENOMEANDO COLUNAS FAIXA DE IDADE
df_6 = df_6.withColumnRenamed('Grandes Regiões, Unidades da Federação e Municípios das Capitais', 'UF')
df_6 = df_6.withColumnRenamed('Media 15 a 17 anos', 'Media_Freq_15_a_17_anos')
df_6 = df_6.withColumnRenamed('média 18 a 24 anos', 'Media_Freq_18_a_24_anos')
df_6 = df_6.withColumnRenamed('média 25 anos ou mais', 'Media_Freq_25_anos_ou_mais')

# df_6.show(1000, truncate=False)

# FILTRO PARA REMOVER REGIÕES E CAPITAIS
df_6 = df_6.filter(
            (df_6.UF != "Brasil") & 
            (df_6.UF != "Norte") & 
            (df_6.UF != "Nordeste") &
            (df_6.UF != "Sudeste") & 
            (df_6.UF != "Sul") &
            (df_6.UF != "Centro-Oeste") &
            (df_6.UF != "Porto Velho") &
            (df_6.UF != "Rio Branco") &
            (df_6.UF != "Manaus") &
            (df_6.UF != "Boa Vista") &
            (df_6.UF != "Macapá") &
            (df_6.UF != "Palmas") &
            (df_6.UF != "São Luiz") &
            (df_6.UF != "Teresina") &
            (df_6.UF != "Fortaleza") &
            (df_6.UF != "Natal") &
            (df_6.UF != "joão Pessoa") &
            (df_6.UF != "Recife") &
            (df_6.UF != "Maceió") &
            (df_6.UF != "Aracaju") &
            (df_6.UF != "Salvador") &
            (df_6.UF != "Belo Horizonte") &
            (df_6.UF != "Vitória") &
            (df_6.UF != "Curitiba") &
            (df_6.UF != "Florianópolis") &
            (df_6.UF != "Porto Alegre") &
            (df_6.UF != "Campo Grande") &
            (df_6.UF != "Cuiabá") &
            (df_6.UF != "Goiânia") &
            (df_6.UF != "Brasilia") &
            (df_6.Media_Freq_15_a_17_anos != 89.58849285882357) & # Rio de Janeiro
            (df_6.Media_Freq_15_a_17_anos != 86.56596333926969) # São Paulo
            )\
            .orderBy('UF', ascending=True)

# # INCREMENTANDO TAXA DE EVASÃO
# df_6 = df_6.withColumn('Evasao_11_a_14_anos', (df_5.Freq_11_a_14_anos - 100)) 
# df_6 = df_6.withColumn('Evasao_15_a_17_anos', (df_5.Freq_11_a_14_anos - 100))
# df_6 = df_6.withColumn('Evasao_18_a_24_anos', (df_5.Freq_11_a_14_anos - 100))
# df_6 = df_6.withColumn('Evasao_25_anos_ou_mais', (df_5.Freq_11_a_14_anos - 100))

# CONVERTENDO VALORES EM PORCENTAGEM
'''https://stackoverflow.com/questions/60673912/how-to-convert-number-into-percentage'''

df_6 = df_6\
.withColumn("Media_Freq_15_a_17_anos", F.concat((F.col("Media_Freq_15_a_17_anos") * 1).cast("int"), F.lit(' %')))\
.withColumn("Media_Freq_18_a_24_anos", F.concat((F.col("Media_Freq_18_a_24_anos") * 1).cast("int"), F.lit(' %')))\
.withColumn("Media_Freq_25_anos_ou_mais", F.concat((F.col("Media_Freq_25_anos_ou_mais") * 1).cast("int"), F.lit(' %')))
# .withColumn("Media_Freq_Escolar", F.concat((F.col("Media_Freq_Escolar") * 1).cast("int"), F.lit(' %')))\
# .withColumn("Evasao_11_a_14_anos", F.concat((F.col("Evasao_11_a_14_anos") * 1).cast("int"), F.lit(' %')))\
# .withColumn("Evasao_15_a_17_anos", F.concat((F.col("Evasao_15_a_17_anos") * 1).cast("int"), F.lit(' %')))\
# .withColumn("Evasao_18_a_24_anos", F.concat((F.col("Evasao_18_a_24_anos") * 1).cast("int"), F.lit(' %')))\
# .withColumn("Evasao_25_anos_ou_mais", F.concat((F.col("Evasao_25_anos_ou_mais") * 1).cast("int"), F.lit(' %')))

# AJUSTANDO EXIBIÇÃO DO DATAFRAME
df_6 = df_6.select('UF', 
                   'Media_Freq_15_a_17_anos', 
                #    'Evasao_11_a_14_anos',
                   'Media_Freq_18_a_24_anos',
                #    'Evasao_15_a_17_anos',
                   'Media_Freq_25_anos_ou_mais',
                #    'Evasao_18_a_24_anos',
                #    'Freq_25_anos_ou_mais',
                #    'Evasao_25_anos_ou_mais',
                #    'Media_Freq_Escolar'
               )

df_6.show(1000)

DATAFRAME 7

In [21]:
# DATAFRAME 7
df_7 = df_7.drop('Grupo_idade')
df_7 = df_7.drop('Unnamed: 0')

df_7 = df_7\
.withColumn("TaxaAnalfabetismo2016", F.concat((F.col("TaxaAnalfabetismo2016") * 1).cast("int"), F.lit(' %')))\
.withColumn("TaxaAnalfabetismo2017", F.concat((F.col("TaxaAnalfabetismo2017") * 1).cast("int"), F.lit(' %')))\
.withColumn("TaxaAnalfabetismo2018", F.concat((F.col("TaxaAnalfabetismo2018") * 1).cast("int"), F.lit(' %')))\
.withColumn("TaxaAnalfabetismo2019", F.concat((F.col("TaxaAnalfabetismo2019") * 1).cast("int"), F.lit(' %')))

df_7.show()

+-------------------+---------------------+---------------------+---------------------+---------------------+
|                 UF|TaxaAnalfabetismo2016|TaxaAnalfabetismo2017|TaxaAnalfabetismo2018|TaxaAnalfabetismo2019|
+-------------------+---------------------+---------------------+---------------------+---------------------+
|           Rondônia|                  6 %|                  7 %|                  6 %|                  6 %|
|               Acre|                 13 %|                 12 %|                 12 %|                 11 %|
|           Amazonas|                  6 %|                  6 %|                  5 %|                  5 %|
|            Roraima|                  6 %|                  6 %|                  6 %|                  5 %|
|               Pará|                  9 %|                  8 %|                  8 %|                  8 %|
|              Amapá|                  5 %|                  5 %|                  6 %|                  5 %|
|         

### CONSULTAS / AGREGANDO DADOS

In [22]:
# VISUALIZAR DATAFRAMES TRATADOS
df_1.show(5, truncate=False)
df_2.show(5, truncate=False)
df_3.show(5, truncate=False)
df_4.show(5, truncate=False)
df_5.show(5, truncate=False)
df_6.show(5, truncate=False)
df_7.show(5, truncate=False)

+--------+---------------+------------+------------+------------+------------+---------+------------------+
|UF      |Despesas2016   |Despesas2017|Despesas2018|Despesas2019|Despesas2020|Variacao%|Previsao2021_Media|
+--------+---------------+------------+------------+------------+------------+---------+------------------+
|Acre    |4.9853469586E8 |4.98534688E8|6.2735085E8 |6.9257018E8 |4.9375328E8 |8.5      |5.7611411E8       |
|Amapá   |4.7532329175E8 |4.75323296E8|4.89227968E8|6.0636634E8 |7.5225248E8 |52.3     |5.6467942E8       |
|Amazonas|1.60473221563E9|1.60473216E9|1.86266893E9|2.1084599E9 |2.10144666E9|60.4     |1.85125286E9      |
|Pará    |2.55174076992E9|2.55174067E9|2.90320026E9|2.94344755E9|2.96655334E9|58.0     |2.78897152E9      |
|Rondônia|8.7487402427E8 |8.7487405E8 |9.0325389E8 |8.5397101E8 |9.990439E8  |-14.7    |9.0540211E8       |
+--------+---------------+------------+------------+------------+------------+---------+------------------+
only showing top 5 rows

+--

NameError: ignored

JOINS

In [ ]:
# AGREGAÇÃO 1 (JOIN 1, 2, 3)

#AGRUPANDO VALORES DOS DATAFRAMES
df_ins2 = df_2.groupBy('UF').sum('Ocorrencias').orderBy('UF')
df_ins2.show(50, truncate=False)

df_ins3 = df_3.groupBy('UF').sum('Vitimas').orderBy('UF')
df_ins3.show(50, truncate=False)

# ------------------------------------------------------------

# JOIN 1
# AGREGA OS VALORES TOTAIS DE OCORRENCIAS E VITIMAS

df_join_1 = df_ins2.join(df_ins3, on=['UF'], how='inner')\
.select('UF', 'sum(Ocorrencias)', 'sum(Vitimas)')\
.orderBy(F.asc('UF'))

df_join_1.show(50, False)

# ------------------------------------------------------------

# JOIN 2
# AGREGA OS VALORES TOTAIS DE OCORRENCIAS, VITIMAS, MEDIA DE FREQ ESCOLAR

df_join_2 = df_join_1.join(df_6, on=['UF'], how='inner')\
.select('UF', 'sum(Ocorrencias)', 'sum(Vitimas)', 'Media_Freq_25_anos_ou_mais')\
.orderBy('UF', ascending=True)

df_join_2.show(50, False)

# ------------------------------------------------------------

# JOIN 3
# AGREGA OS VALORES TOTAIS DE DESPESAS GOVERNAMENTAIS, OCORRENCIAS, VITIMAS, MEDIA DE FREQ. ESCOLAR

df_join_3 = df_join_2.join(df_1, on=['UF'], how='inner')\
.select('UF', 'Despesas2020', 'sum(Ocorrencias)', 'sum(Vitimas)', 'Media_Freq_25_anos_ou_mais')\
.orderBy(F.asc('UF'))

df_join_3\
.orderBy("Despesas2020", ascending=False)\
.show(50, False)

df_join_3.dtypes

+-------------------+----------------+
|UF                 |sum(Ocorrencias)|
+-------------------+----------------+
|Acre               |10569           |
|Alagoas            |36240           |
|Amapá              |9594            |
|Amazonas           |42023           |
|Bahia              |159173          |
|Ceará              |109815          |
|Distrito Federal   |60852           |
|Espírito Santo     |71950           |
|Goiás              |124737          |
|Maranhão           |59384           |
|Mato Grosso        |46055           |
|Mato Grosso do Sul |38831           |
|Minas Gerais       |216193          |
|Paraná             |182637          |
|Paraíba            |23523           |
|Pará               |94224           |
|Pernambuco         |161713          |
|Piauí              |45802           |
|Rio Grande do Norte|50289           |
|Rio Grande do Sul  |208516          |
|Rio de Janeiro     |422633          |
|Rondônia           |33431           |
|Roraima            |1019

[('UF', 'string'),
 ('Despesas2020', 'float'),
 ('sum(Ocorrencias)', 'bigint'),
 ('sum(Vitimas)', 'bigint'),
 ('Media_Freq_25_anos_ou_mais', 'string')]

In [ ]:
# AGREGAÇÃO 2 (JOIN 4)

# JOIN 2
# AGREGA OS VALORES TOTAIS DE OCORRENCIAS E VITIMAS POR ESTADO 

# ------------------------------------------------------------

df_join_4 = df_6.join(df_7, on=['UF'], how='inner')\
.select('UF', 
        'Media_Freq_15_a_17_anos', 
        'Media_Freq_18_a_24_anos', 
        'Media_Freq_25_anos_ou_mais', 
        'TaxaAnalfabetismo2016', 
        'TaxaAnalfabetismo2017', 
        'TaxaAnalfabetismo2018', 
        'TaxaAnalfabetismo2019')\
.orderBy(F.asc('UF'))

df_join_4.show()

+-------------------+-----------------------+-----------------------+--------------------------+---------------------+---------------------+---------------------+---------------------+
|                 UF|Media_Freq_15_a_17_anos|Media_Freq_18_a_24_anos|Media_Freq_25_anos_ou_mais|TaxaAnalfabetismo2016|TaxaAnalfabetismo2017|TaxaAnalfabetismo2018|TaxaAnalfabetismo2019|
+-------------------+-----------------------+-----------------------+--------------------------+---------------------+---------------------+---------------------+---------------------+
|               Acre|                   86 %|                   39 %|                       9 %|                 13 %|                 12 %|                 12 %|                 11 %|
|            Alagoas|                   85 %|                   35 %|                       4 %|                 19 %|                 18 %|                 17 %|                 17 %|
|              Amapá|                   88 %|                   42 %|      

In [ ]:
# AGREGAÇÃO 3 (JOIN 5)

# AGRUPANDO VALORES DOS DATAFRAMES
df_agg_1 = df_2.filter(df_2.Ano == 2020)\
.groupBy('UF').sum('Ocorrencias').alias('Ocorrencias_2020').orderBy('UF')

df_agg_1.show()

# ------------------------------------------------------------

df_agg_2 = df_3.filter(df_3.Ano == 2020)\
.groupBy('UF').sum('Vitimas').alias('Vitimas_2020').orderBy('UF')

df_agg_2.show()

# ------------------------------------------------------------

# AGREGANDO OS VALORES PELO ANO DE 2020

df_join_5 = df_agg_1.join(df_agg_2, on=['UF'], how='inner')\
.select('UF', 'sum(Ocorrencias)', 'sum(Vitimas)')\
.orderBy('UF', ascending=True)

df_join_5.show()

# ------------------------------------------------------------

df_join_2 = df_1.select('UF', 'Despesas2020')

df_join_5 = df_join_5.join(df_join_2, on=['UF'], how='inner')\
.select('UF', 
        'sum(Ocorrencias)', 
        'sum(Vitimas)', 
        'Despesas2020')

df_join_5 = df_join_5\
.withColumnRenamed('sum(Ocorrencias)', 'Ocorrencias_2020')\
.withColumnRenamed('sum(Vitimas)', 'Vitimas_2020')\
.withColumnRenamed('Despesas2020', 'Despesas_Gov_2020')\

df_join_5.show()

+-------------------+----------------+
|                 UF|sum(Ocorrencias)|
+-------------------+----------------+
|               Acre|            1850|
|            Alagoas|            5404|
|              Amapá|             860|
|           Amazonas|            5717|
|              Bahia|           24392|
|              Ceará|           19402|
|   Distrito Federal|            8269|
|     Espírito Santo|           13224|
|              Goiás|           11868|
|           Maranhão|            8356|
|        Mato Grosso|            6466|
| Mato Grosso do Sul|            5937|
|       Minas Gerais|           27002|
|             Paraná|           24872|
|            Paraíba|            4822|
|               Pará|           11824|
|         Pernambuco|           23832|
|              Piauí|            8015|
|Rio Grande do Norte|            7976|
|  Rio Grande do Sul|           25810|
+-------------------+----------------+
only showing top 20 rows

+-------------------+------------+
|  

In [ ]:
# AGREGAÇÃO 4

df_agg_4 = df_4\
.groupBy('UF', 'Ano').avg('Media_Freq_Escolar')\
.alias('Media_Freq_Escolar')\
.orderBy('UF', 'Ano')

df_agg_4.show()

+--------+----+-----------------------+
|      UF| Ano|avg(Media_Freq_Escolar)|
+--------+----+-----------------------+
|    Acre|2016|                   10.6|
|    Acre|2017|                   10.8|
|    Acre|2018|                   10.9|
|    Acre|2019|                   11.1|
| Alagoas|2016|                    9.7|
| Alagoas|2017|                   10.0|
| Alagoas|2018|                   10.4|
| Alagoas|2019|                   10.4|
|   Amapá|2016|                   11.1|
|   Amapá|2017|                   11.2|
|   Amapá|2018|                   11.3|
|   Amapá|2019|                   11.5|
|Amazonas|2016|                   10.5|
|Amazonas|2017|                   10.8|
|Amazonas|2018|                   10.9|
|Amazonas|2019|                   11.2|
|   Bahia|2016|                   10.4|
|   Bahia|2017|                   10.3|
|   Bahia|2018|                   10.4|
|   Bahia|2019|                   10.7|
+--------+----+-----------------------+
only showing top 20 rows



WINDOW FUNCTIONS RANKING

In [ ]:
# ANALISANDO AS OCORRENCIAS DO ESTADO SP EM 2019

df_2_consulta = df_2.groupby('UF', 'TipoCrime', 'Ano')\
.sum('Ocorrencias')\
.filter(
    (df_2.UF == "São Paulo") & 
    (df_2.Ano == 2019))\
.orderBy('TipoCrime')

# --------------------------------------------

# RANKING DAS OCORRENCIAS
w0 = Window.partitionBy(F.col('UF')).orderBy(F.desc('sum(Ocorrencias)'))

df_2_consulta.withColumn('Ranking', F.row_number().over(w0))\
.show(truncate=False)

+---------+-----------------------------------+----+----------------+-------+
|UF       |TipoCrime                          |Ano |sum(Ocorrencias)|Ranking|
+---------+-----------------------------------+----+----------------+-------+
|São Paulo|Furto de veículo                   |2019|84099           |1      |
|São Paulo|Roubo de veículo                   |2019|46513           |2      |
|São Paulo|Estupro                            |2019|11614           |3      |
|São Paulo|Roubo de carga                     |2019|7325            |4      |
|São Paulo|Tentativa de homicídio             |2019|3390            |5      |
|São Paulo|Homicídio doloso                   |2019|2778            |6      |
|São Paulo|Roubo seguido de morte (latrocínio)|2019|191             |7      |
|São Paulo|Lesão corporal seguida de morte    |2019|99              |8      |
|São Paulo|Roubo a instituição financeira     |2019|21              |9      |
+---------+-----------------------------------+----+------------

STRUCT TYPE | UNION BY NAME

In [ ]:
# CRIANDO DATAFRAME VAZIO

esquema = (StructType([
        StructField('UF', StringType(), True),
        StructField('TipoCrime', StringType(), True),
        StructField('Ano', IntegerType(), True),
        StructField('Mes', StringType(), True),
        StructField('SexoVitima', StringType(), True),
        StructField('Vitimas', IntegerType(), True)
    ])
)

df_3_consulta = spark.createDataFrame(data='', schema=esquema)
df_3_consulta.printSchema()
df_3_consulta.show()

# UNINDO DF

df_3_consulta.unionByName(df_3)\
.orderBy('Ano', ascending=False)\
.show(5, truncate=False)

# --------------------------------------------

# EXIBINDO AS QTD. DE VITIMAS DE CRIMES DO ESTADO RJ EM 2020 POR SEXO

df_3_consulta = df_3.groupby('UF', 'TipoCrime', 'Ano', 'SexoVitima')\
.sum('Vitimas')\
.filter(
    (df_3.UF == "Rio de Janeiro") &
    (df_3.Ano == 2020) &
    (df_3.SexoVitima == "Feminino"))\
.orderBy('TipoCrime')

df_3_consulta.show(truncate=False)

df_3_consulta = df_3.groupby('UF', 'TipoCrime', 'Ano', 'SexoVitima')\
.sum('Vitimas')\
.filter(
    (df_3.UF == "Rio de Janeiro") &
    (df_3.Ano == 2020) &
    (df_3.SexoVitima == "Masculino"))\
.orderBy('TipoCrime')

df_3_consulta.show(truncate=False)

df_3_consulta = df_3.groupby('UF', 'TipoCrime', 'Ano', 'SexoVitima')\
.sum('Vitimas')\
.filter(
    (df_3.UF == "Rio de Janeiro") &
    (df_3.Ano == 2020) &
    (df_3.SexoVitima == "Sexo NI"))\
.orderBy('TipoCrime')

df_3_consulta.show(truncate=False)

root
 |-- UF: string (nullable = true)
 |-- TipoCrime: string (nullable = true)
 |-- Ano: integer (nullable = true)
 |-- Mes: string (nullable = true)
 |-- SexoVitima: string (nullable = true)
 |-- Vitimas: integer (nullable = true)

+---+---------+---+---+----------+-------+
| UF|TipoCrime|Ano|Mes|SexoVitima|Vitimas|
+---+---------+---+---+----------+-------+
+---+---------+---+---+----------+-------+

+----+----------------+----+---------+----------+-------+
|UF  |TipoCrime       |Ano |Mes      |SexoVitima|Vitimas|
+----+----------------+----+---------+----------+-------+
|Acre|Homicídio doloso|2021|janeiro  |Feminino  |1      |
|Acre|Homicídio doloso|2021|janeiro  |Masculino |13     |
|Acre|Homicídio doloso|2021|janeiro  |Sexo NI   |0      |
|Acre|Homicídio doloso|2021|fevereiro|Feminino  |4      |
|Acre|Homicídio doloso|2021|fevereiro|Masculino |12     |
+----+----------------+----+---------+----------+-------+
only showing top 5 rows

+--------------+------------------------------

HEADER | AGGREGATE | SUM

In [ ]:
# ALTERANDO CABEÇALHO

file_path_1 = 'gs://data_lake_ingest_data/2_temp/temp_pandas_despesas_normalizado.csv'

fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)
with fs.open(file_path_1):
    data = pd.read_csv(file_path_1,
                       sep=',', 
                       encoding='UTF-8',
                       header=None)

esquema=['UF', 
         'Desp_2016', 
         'Desp_2017', 
         'Desp_2018', 
         'Desp_2019', 
         'Desp_2020', 
         'Variacao_%', 
         'Previsao_2021_Media']

df_1_consulta = spark.createDataFrame(data, schema=esquema, )

df_1_consulta.show(5, truncate=False)

# --------------------------------------------

# AGREGANDO O TOTAL DE DESPESAS GOVERNAMENTAIS DO PARANA REFERENTES AOS ULTIMOS 3 ANOS

df_1_consulta.select('UF', 'Desp_2018', 'Desp_2019', 'Desp_2020')\
.filter((df_1_consulta.UF == 'Paraná'))\
.groupBy('UF', 'Desp_2018', 'Desp_2019', 'Desp_2020')\
.agg(F.sum(df_1_consulta.Desp_2018 + 
           df_1_consulta.Desp_2019 + 
           df_1_consulta.Desp_2020).alias('PR_Despesas_Totais_Milhoes'))\
.show()

+--------+------------+------------+------------+------------+------------+----------+-------------------+
|UF      |Desp_2016   |Desp_2017   |Desp_2018   |Desp_2019   |Desp_2020   |Variacao_%|Previsao_2021_Media|
+--------+------------+------------+------------+------------+------------+----------+-------------------+
|UF      |Despesas2016|Despesas2017|Despesas2018|Despesas2019|Despesas2020|Variacao% |Previsao2021|Media |
|Acre    |498.535     |568.361     |627.351     |692.57      |493.753     |8.5       |576.114            |
|Amapá   |475.323     |500.227     |489.228     |606.366     |752.252     |52.3      |564.6792           |
|Amazonas|1604.732    |1578.957    |1862.669    |2108.46     |2101.447    |60.4      |1851.253           |
|Pará    |2551.741    |2579.916    |2903.2      |2943.448    |2966.553    |58.0      |2788.9716          |
+--------+------------+------------+------------+------------+------------+----------+-------------------+
only showing top 5 rows

+------+----

FILTERS

In [ ]:
# FILTRANDO INTERVALOS

# IGUAL A 90% E MAIOR QUE 20 %
df_6.filter(
    (df_6.Media_Freq_15_a_17_anos == '90 %') & 
    (df_6.Media_Freq_15_a_17_anos > '20 %'))\
.select('UF', 'Media_Freq_15_a_17_anos')\
.orderBy(F.desc('Media_Freq_15_a_17_anos'))\
.show(5, truncate=False)

# IGUAL E MENOR A 40 % OU IGUAL E MAIOR QUE 20 %
df_6.filter(
    (df_6.Media_Freq_18_a_24_anos <= '40 %') |
    (df_6.Media_Freq_18_a_24_anos >= '20 %'))\
.select('UF', 'Media_Freq_18_a_24_anos')\
.orderBy(F.desc('Media_Freq_18_a_24_anos'))\
.show(5, truncate=False)

# IGUAL E MENOR QUE 20 % OU MAIOR QUE 8 %
df_6.filter(
    (df_6.Media_Freq_25_anos_ou_mais <= '20 %') |
    (df_6.Media_Freq_25_anos_ou_mais > '8 %'))\
.select('UF', 'Media_Freq_25_anos_ou_mais')\
.orderBy(F.desc('Media_Freq_25_anos_ou_mais'))\
.show(5, truncate=False)

+--------+-----------------------+
|UF      |Media_Freq_15_a_17_anos|
+--------+-----------------------+
|Amazonas|90 %                   |
|Roraima |90 %                   |
|Pará    |90 %                   |
|Paraíba |90 %                   |
|Bahia   |90 %                   |
+--------+-----------------------+
only showing top 5 rows

+----------------+-----------------------+
|UF              |Media_Freq_18_a_24_anos|
+----------------+-----------------------+
|Espírito Santo  |47 %                   |
|Distrito Federal|44 %                   |
|Paraíba         |44 %                   |
|Brasília        |44 %                   |
|Santa Catarina  |44 %                   |
+----------------+-----------------------+
only showing top 5 rows

+----+--------------------------+
|UF  |Media_Freq_25_anos_ou_mais|
+----+--------------------------+
|Acre|9 %                       |
+----+--------------------------+



### EXPORTANDO DATAFRAMES

DATAFRAME 1

In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
df_1 = df_1.toPandas()
df_1.to_csv('temp_pyspark_tabela_despesas_gov_uf.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_tabela_despesas_gov_uf.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_tabela_despesas_gov_uf.csv')

DATAFRAME 3

In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
df_3 = df_3.toPandas()
df_3.to_csv('temp_pyspark_tabela_vitimas_sexo.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_tabela_vitimas_sexo.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_tabela_vitimas_sexo.csv')

DATAFRAME 6

In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
df_6 = df_6.toPandas()
df_6.to_csv('temp_pyspark_tabela_frequencia_escolar_normalizado.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_tabela_frequencia_escolar_normalizado.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_tabela_frequencia_escolar_normalizado.csv')

DATAFRAME 7

In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
df_7 = df_7.toPandas()
df_7.to_csv('temp_pyspark_tabela_TaxaAnalfabetismo_normalizado.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_tabela_TaxaAnalfabetismo_normalizado.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_tabela_TaxaAnalfabetismo_normalizado.csv')

JOINT AGREGAÇÃO 1

In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
df_join_1 = df_join_1.toPandas()
df_join_1.to_csv('temp_pyspark_agg_1_General_Data.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_agg_1_General_Data.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_agg_1_General_Data.csv')

JOIN AGREGAÇÃO 2

In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
df_join_4 = df_join_4.toPandas()
df_join_4.to_csv('temp_pyspark_agg_2_Freq_Escolar_Taxa_Analfa.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_agg_2_Freq_Escolar_Taxa_Analfa.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_agg_2_Freq_Escolar_Taxa_Analfa.csv')

JOIN AGREGAÇÃO 3

In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
df_join_5 = df_join_5.toPandas()
df_join_5.to_csv('temp_pyspark_agg_3_general_data_2020.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_agg_3_general_data_2020.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_agg_3_general_data_2020.csv')

AGREGAÇÃO 4 (DATAFRAME 4)

In [ ]:
# TRANSFORMARDO DATAFRAME PYSPAK EM PANDAS
df_agg_4 = df_agg_4.toPandas()
df_agg_4.to_csv('temp_pyspark_agg_4_freq_escolar_normalizada.csv')

BUCKET_NAME= "data_lake_ingest_data"
storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

fileout = "2_temp/temp_pyspark_agg_4_freq_escolar_normalizada.csv"
destination_blob = bucket.blob(fileout)
destination_blob.upload_from_filename('/content/temp_pyspark_agg_4_freq_escolar_normalizada.csv')

In [ ]:
!pip install pyspark
!pip install gcsfs
!pip install pandas

# SPARKSQL

### INSTALANDO DEPENDECIAS

In [ ]:
!pip install pyspark
!pip install gcsfs
!pip install pandas

### IMPORTANDO BIBLIOTECAS, ABRINDO SPARKSESSION E CONFIGURANDO CHAVE DE SERVIÇO

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from google.cloud import storage

import pyspark
import pyspark.sql.functions as F
import gcsfs

In [ ]:
spark = SparkSession.builder\
.master('local')\
.appName('Projeto_Final')\
.config('spark.ui.enable', 'true')\
.config('spark.ui.port', '4050')\
.getOrCreate()

spark

serviceaccount = '/content/soulcode-projeto-final-4b88bea6e07a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

spark

### LENDO DATAFRAME, CRIANDO VISUALIZAÇÃO TEMPORARIA E ANALISANDO DADOS

DATAFRAME 1

In [30]:
file_path = ['',
             'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencia_vitimas_porAnoEstado.csv',
             'gs://data_lake_ingest_data/2_temp/temp_pandas_vitimas_normalizado.json',
             'gs://data_lake_ingest_data/2_temp/temp_pyspark_agg_2_Freq_Escolar_Taxa_Analfa.csv']

# Lendo dataframes dinâmicamente
for i in range(len(file_path)):

    fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)

    if i != 0 and i < 5:
        if ".csv" in file_path[i]:
            with fs.open(file_path[i]):
                data = pd.read_csv(file_path[i], sep=',', encoding='UTF-8', header=0)

            # Criando visualização temporária SparkSQL
            spark.createDataFrame(data).createOrReplaceTempView('df_'+str(i))
        
        elif ".json" in file_path[i]:
            with fs.open(file_path[i]):
                data = pd.read_json(file_path[i], encoding='UTF-8')

            # Criando visualização temporária SparkSQL
            spark.createDataFrame(data).createOrReplaceTempView('df_'+str(i))

    else:
        continue

# Verificando tipos de dados vindos do PySpark
spark.sql(''' DESCRIBE df_1 ''').show()
spark.sql(''' DESCRIBE df_2 ''').show()
spark.sql(''' DESCRIBE df_3 ''').show()

# Visualizando tabelas
spark.sql('''
        SELECT *
        FROM df_1
        LIMIT 10
        '''    
        ).show()

spark.sql('''
        SELECT *
        FROM df_2
        LIMIT 10
        '''    
        ).show()

spark.sql('''
        SELECT *
        FROM df_3
        LIMIT 10
        '''    
        ).show()

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|         UF|   string|   null|
|        Ano|   bigint|   null|
|  TipoCrime|   string|   null|
|Ocorrencias|   bigint|   null|
|    Vitimas|   bigint|   null|
+-----------+---------+-------+

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|        UF|   string|   null|
| TipoCrime|   string|   null|
|       Ano|   bigint|   null|
|       Mes|   string|   null|
|SexoVitima|   string|   null|
|   Vitimas|   bigint|   null|
+----------+---------+-------+

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|          Unnamed: 0|   bigint|   null|
|                  UF|   string|   null|
|Media_Freq_15_a_1...|   string|   null|
|Media_Freq_18_a_2...|   string|   null|
|Media_Freq_25_ano...|   string|   null|
|TaxaAnalfabetismo...|   string|   null|
|TaxaAnalfabetismo...|   string

### CONSULTAS

Total de ocorrencias criminais por Estado (nos anos de 2016 a 2021)



In [ ]:
spark.sql(
'''
SELECT UF, SUM(Ocorrencias) AS Total_Ocorrencias_por_estado
FROM df_1
GROUP BY UF
ORDER BY UF ASC
'''
).show()

In [ ]:
spark.sql(
  
'''
SELECT TipoCrime, SexoVitima, SUM(Vitimas) AS SomaVitimas
FROM df_3
GROUP BY TipoCrime,SexoVitima
ORDER BY TipoCrime
'''

).show(10)